In [71]:
import pandas as pd
import csv
import glob
import os

In [132]:
dataset = pd.read_csv('C:/Users/LG/Desktop/CPS dataset/merge.csv')
dataset.columns

Index(['frame.number', 'ip.src', 'ip.dst', 'eth.src', 'eth.dst',
       'arp.src.hw_mac', 'arp.dst.hw_mac', 'icmp', 'ip.proto', 'tcp.srcport',
       'tcp.dstport', 'tcp.len', 'tcp.seq', 'tcp.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.stream',
       'label'],
      dtype='object')

In [133]:
#사용하지 않을 속성 제거
dataset.pop('ip.src')
dataset.pop('ip.dst')
dataset.pop('eth.src')
dataset.pop('eth.dst')
dataset.pop('arp.src.hw_mac')
dataset.pop('arp.dst.hw_mac')
dataset.pop('icmp')
dataset.pop('ip.proto')
dataset.pop('frame.number')
dataset.pop('tcp.stream')

#결측치 치환, 형변환
x=dataset.fillna(value=0)
x=x.astype('float32')

In [146]:
#dataframe shuffle
from sklearn.utils import shuffle
x=shuffle(x)

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_len=len(x.columns)-1
y_len=len(x.columns)-1
X=x.iloc[:,0:X_len]
y=x.iloc[:,y_len]

k=20 #number or fold for cross validation
num_val_samples=len(X)//20
classifier = DecisionTreeClassifier(criterion = 'entropy')

#confusion matrix for each fold
accuracy_s=[]
precision_rate_s=[]
recall_rate_s=[]
f1_score_s=[]

In [147]:
for i in range(k): #k-fold
    print('처리중인 폴드 #',i+1)
    
    #Ready for Test Set
    val_data=X[i*num_val_samples:(i+1)*num_val_samples]
    val_targets=y[i*num_val_samples:(i+1)*num_val_samples]
    
    #Ready for Training Set
    partial_train_data=np.concatenate([X[:i*num_val_samples],
    X[(i+1)*num_val_samples:]],axis=0)
    partial_train_targets=np.concatenate([y[:i*num_val_samples],
    y[(i+1)*num_val_samples:]],axis=0)
    
    #Feature Scaling
    partial_train_data=sc.fit_transform(partial_train_data)
    val_data=sc.transform(val_data)
    
    # Fitting the decision tree to the Training set
    classifier.fit(partial_train_data, partial_train_target)
    y_pred=classifier.predict(val_data)
    y_pred=(y_pred>0.5)
    
    #confusion matrix for each fold
    cm=confusion_matrix(val_targets, y_pred)
    
    accuracy=(cm[0][0]+cm[1][1])/(cm[0][0]+cm[1][1]+cm[1][0]+cm[0][1])
    precision_rate=cm[1][1]/(cm[1][1]+cm[0][1])
    recall_rate=cm[1][1]/(cm[1][1]+cm[1][0])
    f1_score=cm[1][1]/(cm[1][1]+(cm[1][0]+cm[0][1])/2)
    
    accuracy_s.append(accuracy)
    precision_rate_s.append(precision_rate)
    recall_rate_s.append(recall_rate)
    f1_score_s.append(f1_score)
    
    #confusion matrix for each fold
    print('#', i+1,'폴드의 confusion matrix는 다음과 같습니다')
    print('Confusion Matrix : \n',cm)
    print()

처리중인 폴드 # 1
# 1 폴드의 confusion matrix는 다음과 같습니다
Confusion Matrix : 
 [[ 641 1171]
 [1087 1440]]

처리중인 폴드 # 2
# 2 폴드의 confusion matrix는 다음과 같습니다
Confusion Matrix : 
 [[ 629 1150]
 [1117 1443]]

처리중인 폴드 # 3
# 3 폴드의 confusion matrix는 다음과 같습니다
Confusion Matrix : 
 [[ 639 1136]
 [1112 1452]]

처리중인 폴드 # 4
# 4 폴드의 confusion matrix는 다음과 같습니다
Confusion Matrix : 
 [[ 639 1154]
 [1083 1463]]

처리중인 폴드 # 5
# 5 폴드의 confusion matrix는 다음과 같습니다
Confusion Matrix : 
 [[ 662 1166]
 [1076 1435]]

처리중인 폴드 # 6
# 6 폴드의 confusion matrix는 다음과 같습니다
Confusion Matrix : 
 [[ 645 1191]
 [1093 1410]]

처리중인 폴드 # 7
# 7 폴드의 confusion matrix는 다음과 같습니다
Confusion Matrix : 
 [[ 661 1149]
 [1068 1461]]

처리중인 폴드 # 8
# 8 폴드의 confusion matrix는 다음과 같습니다
Confusion Matrix : 
 [[ 641 1176]
 [1079 1443]]

처리중인 폴드 # 9
# 9 폴드의 confusion matrix는 다음과 같습니다
Confusion Matrix : 
 [[ 639 1116]
 [1089 1495]]

처리중인 폴드 # 10
# 10 폴드의 confusion matrix는 다음과 같습니다
Confusion Matrix : 
 [[ 681 1128]
 [1072 1458]]

처리중인 폴드 # 11
# 11 폴드의 confusion matrix

In [148]:
print('accuracy_s:', np.mean(accuracy_s))
print('precision_rate_s:', np.mean(precision_rate_s))
print('recall_rate_s:', np.mean(recall_rate_s))
print('f1_score_s:', np.mean(f1_score_s))

accuracy_s: 0.4839594376584467
precision_rate_s: 0.5562712820132606
recall_rate_s: 0.5719857454432113
f1_score_s: 0.5639933117545554


In [149]:
y_hat = classifier.fit(partial_train_data, partial_train_targets).predict(val_data)
from sklearn import metrics

print(metrics.classification_report(val_targets, y_pred),"\n")
print(metrics.confusion_matrix(val_targets, y_pred),"\n")

              precision    recall  f1-score   support

         0.0       0.35      0.35      0.35      1772
         1.0       0.55      0.56      0.56      2567

    accuracy                           0.47      4339
   macro avg       0.45      0.45      0.45      4339
weighted avg       0.47      0.47      0.47      4339
 

[[ 617 1155]
 [1132 1435]] 

